In [1]:
pip install numpy torch qutip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from qutip import Qobj, ket2dm, basis  # only for generating initial qubit states
import os
import torch
from google.cloud import storage


In [12]:
def generate_mixed_state_vectors(n_samples):
    vectors = []
    for _ in range(n_samples):
        theta = np.arccos(2*np.random.rand() - 1)
        phi = 2*np.pi*np.random.rand()
        psi = np.cos(theta/2)*basis(2,0) + np.exp(1j*phi)*basis(2,1)
        pure_dm = ket2dm(psi)
        
        p = np.random.rand()
        if np.random.rand() > 0.5:
            mixed_dm = p*pure_dm + (1-p)*ket2dm(basis(2,1))
        else:
            mixed_dm = p*pure_dm + (1-p)*ket2dm(basis(2,0))
        
        # Convert to 4-number real vector
        vec = np.array([
            mixed_dm[0,0].real,
            mixed_dm[0,1].real,
            mixed_dm[0,1].imag,
            mixed_dm[1,1].real
        ], dtype=np.float32)
        vectors.append(vec)
    return np.stack(vectors, axis=0)


In [13]:
import math
import torch.nn.functional as F

def sinusoidal_embedding(t, dim):
    """
    t: tensor shape (B,1) or (B,) with integer timesteps (0..T-1) normalized to [0,1]
    returns: (B, dim)
    """
    if t.dim() == 1:
        t = t.unsqueeze(-1)
    half = dim // 2
    freqs = torch.exp(-math.log(10000.0) * torch.arange(0, half, dtype=torch.float32) / half).to(t.device)
    args = t.float() * freqs[None, :]
    emb = torch.cat([torch.sin(args), torch.cos(args)], dim=-1)
    if dim % 2 == 1:
        emb = F.pad(emb, (0,1), "constant", 0.0)
    return emb

class QubitDenoiser(nn.Module):
    def __init__(self, hidden_dim=128, time_dim=32):
        super().__init__()
        self.time_dim = time_dim
        self.time_mlp = nn.Sequential(
            nn.Linear(time_dim, time_dim),
            nn.ReLU(),
            nn.Linear(time_dim, time_dim)
        )
        self.net = nn.Sequential(
            nn.Linear(4 + time_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 4)
        )

    def forward(self, x, t):
        # t: tensor of shape (B,) or (B,1) with values in [0,1] (or normalized step index)
        t_emb = sinusoidal_embedding(t, self.time_dim)
        t_emb = self.time_mlp(t_emb)
        inp = torch.cat([x, t_emb], dim=-1)
        return self.net(inp)


In [14]:
def forward_diffusion_multistep(vector_data, original_vectors, steps=5, device='cpu'):
    vector_data = vector_data.to(device)
    original_vectors = original_vectors.to(device)
    next_vectors = vector_data.clone()
    N = next_vectors.shape[0]
    
    sqrt2_inv = 1.0 / torch.sqrt(torch.tensor(2.0, dtype=torch.float32, device=device))
    U_clone = torch.tensor([[1,0,0,0],
                            [0,sqrt2_inv,sqrt2_inv,0],
                            [0,sqrt2_inv,-sqrt2_inv,0],
                            [0,0,0,1]], dtype=torch.complex64, device=device)
    U_clone_conj = U_clone.conj().T

    fidelity_over_steps = []

    for step in range(steps):
        rho = torch.zeros((N,2,2), dtype=torch.complex64, device=device)
        rho[:,0,0] = next_vectors[:,0]
        rho[:,0,1] = next_vectors[:,1] + 1j*next_vectors[:,2]
        rho[:,1,0] = next_vectors[:,1] - 1j*next_vectors[:,2]
        rho[:,1,1] = next_vectors[:,3]

        ancilla = torch.zeros((2,2), dtype=torch.complex64, device=device)
        ancilla[0,0] = 1.0
        rho_joint = torch.zeros((N,4,4), dtype=torch.complex64, device=device)
        rho_joint[:,0:2,0:2] = rho * ancilla[0,0]

        rho_cloned = torch.matmul(torch.matmul(U_clone.expand(N,4,4), rho_joint), U_clone_conj.expand(N,4,4))
        clone1 = rho_cloned[:,0:2,0:2]
        clone2 = rho_cloned[:,2:4,2:4]

        weights = torch.rand(N,1,1, device=device)
        next_rho = weights * clone1 + (1 - weights) * clone2

        # Back to 4-number vectors
        next_vectors[:,0] = next_rho[:,0,0].real
        next_vectors[:,1] = next_rho[:,0,1].real
        next_vectors[:,2] = next_rho[:,0,1].imag
        next_vectors[:,3] = next_rho[:,1,1].real

        # Fidelity proxy
        rho_orig = torch.zeros((N,2,2), dtype=torch.complex64, device=device)
        rho_orig[:,0,0] = original_vectors[:,0]
        rho_orig[:,0,1] = original_vectors[:,1] + 1j*original_vectors[:,2]
        rho_orig[:,1,0] = original_vectors[:,1] - 1j*original_vectors[:,2]
        rho_orig[:,1,1] = original_vectors[:,3]

        avg_fid = torch.mean(torch.real(torch.einsum('nij,nji->n', rho_orig, next_rho)))
        fidelity_over_steps.append(avg_fid.item())

    return next_vectors, fidelity_over_steps

In [15]:
def fidelity_loss(pred_vectors, target_vectors):
    N = pred_vectors.shape[0]
    rho_pred = torch.zeros((N,2,2), dtype=torch.complex64, device=pred_vectors.device)
    rho_pred[:,0,0] = pred_vectors[:,0]
    rho_pred[:,0,1] = pred_vectors[:,1] + 1j*pred_vectors[:,2]
    rho_pred[:,1,0] = pred_vectors[:,1] - 1j*pred_vectors[:,2]
    rho_pred[:,1,1] = pred_vectors[:,3]

    rho_target = torch.zeros((N,2,2), dtype=torch.complex64, device=pred_vectors.device)
    rho_target[:,0,0] = target_vectors[:,0]
    rho_target[:,0,1] = target_vectors[:,1] + 1j*target_vectors[:,2]
    rho_target[:,1,0] = target_vectors[:,1] - 1j*target_vectors[:,2]
    rho_target[:,1,1] = target_vectors[:,3]

    avg_fid = torch.mean(torch.real(torch.einsum('nij,nji->n', rho_target, rho_pred)))
    return 1 - avg_fid


In [16]:
def upload_to_gcs(local_file, bucket_name, blob_name):
    client = storage.Client()  # assumes your GCP credentials are set
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_file)
    print(f"Uploaded {local_file} to gs://{bucket_name}/{blob_name}")

In [17]:
def train_diffusion_with_gcs(model, vectors, bucket_name, gcs_folder="models",
                             epochs=5, batch_size=128, steps=5, lr=1e-3):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    dataset = torch.tensor(vectors, dtype=torch.float32)
    N = dataset.shape[0]
    split = int(0.8 * N)
    train_data = dataset[:split]
    test_data = dataset[split:]
    
    T = 5  # your noise steps

    for epoch in range(epochs):
        permutation = torch.randperm(train_data.shape[0])
        epoch_loss = 0.0

        for i in range(0, train_data.shape[0], batch_size):
            optimizer.zero_grad()

            idx = permutation[i:i+batch_size]
            batch = train_data[idx].to(device)

        # 1️⃣ Sample a random t (same t for whole batch is fine)
            t_scalar = torch.randint(1, T + 1, (1,), device=device).item()

        # 2️⃣ Apply t steps of forward diffusion
            noisy_batch, _ = forward_diffusion_multistep(
                batch, batch, steps=t_scalar, device=device
            )

        # 3️⃣ Model receives normalized timestep in [0,1]
            t_input = torch.full(
                (noisy_batch.size(0),),
                float(t_scalar) / T,
                device=device
            )

        # 4️⃣ Predict clean state x0
            pred = model(noisy_batch, t_input)

        # 5️⃣ Compute loss
            loss = fidelity_loss(pred, batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.6f}")
        # ----------------------
        # Save checkpoint locally and upload to GCS
        # ----------------------
        local_checkpoint = f"qubit_diffusion_epoch{epoch+1}.pt"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, local_checkpoint)

        gcs_blob_name = os.path.join(gcs_folder, f"qubit_diffusion_epoch{epoch+1}.pt")
        upload_to_gcs(local_checkpoint, bucket_name, gcs_blob_name)
        os.remove(local_checkpoint)  # optional: remove local file to save space


In [18]:
import torch

def test_diffusion_model(model, test_vectors, steps=5, device=None, return_per_sample=False):
    """
    Test a trained diffusion model on the test set.
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    model.eval()
    
    test_data = test_vectors.to(device)
    T = steps  # same meaning as during training
    
    with torch.no_grad():
        # 1️⃣ Forward diffusion with T steps
        noisy_test, _ = forward_diffusion_multistep(
            test_data, test_data, steps=T, device=device
        )
        
        # 2️⃣ Prepare timestep input (normalized to [0,1])
        t_input = torch.full(
            (noisy_test.size(0),),
            float(T) / T,   # = 1.0 always for max-noise test
            device=device
        )
        
        # 3️⃣ Denoise using the time-conditioned model
        pred_test = model(noisy_test, t_input)
        
        # 4️⃣ Convert predicted 4-vector → 2x2 density matrix
        N = test_data.shape[0]
        rho_pred = torch.zeros((N,2,2), dtype=torch.complex64, device=device)
        rho_pred[:,0,0] = pred_test[:,0]
        rho_pred[:,0,1] = pred_test[:,1] + 1j*pred_test[:,2]
        rho_pred[:,1,0] = pred_test[:,1] - 1j*pred_test[:,2]
        rho_pred[:,1,1] = pred_test[:,3]

        # 5️⃣ Ground-truth density matrices
        rho_target = torch.zeros((N,2,2), dtype=torch.complex64, device=device)
        rho_target[:,0,0] = test_data[:,0]
        rho_target[:,0,1] = test_data[:,1] + 1j*test_data[:,2]
        rho_target[:,1,0] = test_data[:,1] - 1j*test_data[:,2]
        rho_target[:,1,1] = test_data[:,3]

        # 6️⃣ Fidelity-like overlap
        per_sample_fidelity = torch.real(torch.einsum('nij,nji->n', rho_target, rho_pred))
        avg_fidelity = torch.mean(per_sample_fidelity).item()

    if return_per_sample:
        return avg_fidelity, per_sample_fidelity.cpu().numpy()
    else:
        return avg_fidelity


In [19]:
if __name__ == "__main__":
    import torch
    import os

    # ----------------------
    # 1️⃣ Generate dataset
    # ----------------------
    n_samples = 2000  # total qubit states
    vectors = generate_mixed_state_vectors(n_samples)
    dataset = torch.tensor(vectors, dtype=torch.float32)

    # Split 80/20 train/test
    split = int(0.8 * n_samples)
    train_vectors = dataset[:split]
    test_vectors = dataset[split:]

    print(f"Training samples: {train_vectors.shape[0]}, Test samples: {test_vectors.shape[0]}")

    # ----------------------
    # 2️⃣ Initialize model
    # ----------------------
    model = QubitDenoiser()

    # ----------------------
    # 3️⃣ Define GCS bucket and folder
    # ----------------------
    bucket_name = "instr-cs795-fall25-hqin-1-kseek001" 
    gcs_folder = "qubit_diffusion_checkpoints"

    # ----------------------
    # 4️⃣ Train model with GCS checkpointing
    # ----------------------
    train_diffusion_with_gcs(
        model=model,
        vectors=vectors,
        bucket_name=bucket_name,
        gcs_folder=gcs_folder,
        epochs=10,
        batch_size=128,
        steps=5,
        lr=1e-3
    )

    # ----------------------
    # 5️⃣ Test the trained model
    # ----------------------
    avg_fid, per_sample_fid = test_diffusion_model(
        model=model,
        test_vectors=test_vectors,
        steps=5,
        return_per_sample=True
    )

    print(f"\nFinal Test Set Average Fidelity: {avg_fid:.4f}")


Training samples: 1600, Test samples: 400
Using device: cuda
Epoch 1/10 - Loss: 9.698489
Uploaded qubit_diffusion_epoch1.pt to gs://instr-cs795-fall25-hqin-1-kseek001/qubit_diffusion_checkpoints/qubit_diffusion_epoch1.pt
Epoch 2/10 - Loss: -13.214288
Uploaded qubit_diffusion_epoch2.pt to gs://instr-cs795-fall25-hqin-1-kseek001/qubit_diffusion_checkpoints/qubit_diffusion_epoch2.pt
Epoch 3/10 - Loss: -134.561517
Uploaded qubit_diffusion_epoch3.pt to gs://instr-cs795-fall25-hqin-1-kseek001/qubit_diffusion_checkpoints/qubit_diffusion_epoch3.pt
Epoch 4/10 - Loss: -665.659897
Uploaded qubit_diffusion_epoch4.pt to gs://instr-cs795-fall25-hqin-1-kseek001/qubit_diffusion_checkpoints/qubit_diffusion_epoch4.pt
Epoch 5/10 - Loss: -2543.134743
Uploaded qubit_diffusion_epoch5.pt to gs://instr-cs795-fall25-hqin-1-kseek001/qubit_diffusion_checkpoints/qubit_diffusion_epoch5.pt
Epoch 6/10 - Loss: -8123.334503
Uploaded qubit_diffusion_epoch6.pt to gs://instr-cs795-fall25-hqin-1-kseek001/qubit_diffusion_c